In [1]:
import os
from pathlib import Path
import pandas as pd

In [2]:
PROJECT_ROOT = Path(os.getcwd()).resolve().parents[0]

In [ ]:
df = pd.read_excel(PROJECT_ROOT / "data/raw/nb de contrats actifs  CDI et  KWh 2007 2012.xlsx")

In [56]:
df.head()

,Unnamed: 0,Unnamed: 1,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9,...,Unnamed: 152,Unnamed: 153,Unnamed: 154,Unnamed: 155,Unnamed: 156,Unnamed: 157,Unnamed: 158,Unnamed: 159,Unnamed: 160,Unnamed: 161
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,NaN,NaN,Année civile,Année civile/mois,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NB_CLIENTS,NB_CLIENTS,NB_CLIENTS,NB_CLIENTS,NB_CLIENTS,NB_CLIENTS,...,KWh,KWh,KWh,KWh,KWh,KWh,KWh,KWh,KWh,KWh
3,NaN,NaN,NaN,NaN,2007,2007,2007,2007,2007,2007,...,2012,2012,2012,2012,2012,2012,2012,2012,2012,Résultat global
4,DR_DP_AS,Type d'usage,Clients,usage,01.2007,02.2007,03.2007,04.2007,05.2007,06.2007,...,05.2012,06.2012,07.2012,08.2012,09.2012,10.2012,11.2012,12.2012,Résultat,Résultat


In [6]:
import sqlite3

In [7]:
db_orig = sqlite3.connect(PROJECT_ROOT / "data/ONEE_Regional_COMPLETE.db")

In [18]:
df_regional_orig = pd.read_sql_query("SELECT * FROM monthly_data", db_orig)

In [ ]:
db_orig.close()

In [20]:
df_regional_orig.columns

Index(['Classe', 'Activity', 'City', 'Year', 'Month', 'Nbr Clients', 'MWh',
       'kDH', 'tempmax', 'tempmin', 'temp', 'precip', 'Region'],
      dtype='object')

In [33]:
def extract_city(s):
    return s.split(':')[-1].strip().split()[-1]
unique = set()
for s in df.iloc[5:]["Unnamed: 0"]:
    city = extract_city(s)
    if city not in unique:
        unique.add(city)

In [40]:
unique

{'AGADIR',
 'CASA',
 'ERRACHIDIA',
 'FES',
 'LAAYOUNE',
 'MARRAKECH',
 'MELLAL',
 'OUJDA',
 'RABAT',
 'TANGER'}

In [39]:
df_regional_orig["City"].unique()

array(['agadir', 'beni_mellal', 'casablanca', 'errachidia', 'fes',
       'laayoune', 'marrakech', 'oujda', 'rabat', 'tanger'], dtype=object)

In [77]:
df = df.drop(index= [0,1,3], columns = ["Unnamed: 1", "Unnamed: 2"]).reset_index(drop=True)

In [78]:
df.columns = df.iloc[0].fillna('') + '__' + df.iloc[1].fillna('')

In [79]:
df.columns = df.columns.str.strip('_')
df.columns = df.columns.str.strip()
df = df.drop([0, 1])

In [80]:
activity_mapping = {
  ' ADMINISTRATIF ': 'Administratif',
  'Ménages': 'Menages',
  'FMA': 'Force_Motrice_Agricole',
  'FMI': 'Force_Motrice_Industrielle',
  'PATENTE': 'Patentes',
  'PUBLIC': 'Public',
  'MTG': 'General',
  'MTV': 'Vert'
}

In [86]:
df["usage"] = df["usage"].replace(activity_mapping)

In [87]:
df["usage"].unique()

array(['Administratif', 'Menages', 'Force_Motrice_Agricole',
       'Force_Motrice_Industrielle', 'Patentes', 'Public', 'General',
       'Vert'], dtype=object)

In [90]:
city_mapping = {
    'AGADIR': 'agadir',
    'CASA': 'casablanca',
    'ERRACHIDIA': 'errachidia',
    'FES': 'fes',
    'LAAYOUNE': 'laayoune',
    'MARRAKECH': 'marrakech',
    'MELLAL': 'beni_mellal',
    'OUJDA': 'oujda',
    'RABAT': 'rabat',
    'TANGER': 'tanger'
}

In [93]:
df["DR_DP_AS"] = df["DR_DP_AS"].apply(lambda s: city_mapping[extract_city(s)])

In [96]:
df = df.groupby(["DR_DP_AS","usage"], as_index = False).sum()

C:\Users\Soufiane.AITELAOUAD\AppData\Local\Temp\ipykernel_21248\2714433981.py:1: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df = df.groupby(["DR_DP_AS","usage"], as_index = False).sum()
C:\Users\Soufiane.AITELAOUAD\AppData\Local\Temp\ipykernel_21248\2714433981.py:1: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df = df.groupby(["DR_DP_AS","usage"], as_index = False).sum()


In [97]:
df

,DR_DP_AS,usage,NB_CLIENTS__01.2007,NB_CLIENTS__02.2007,NB_CLIENTS__03.2007,NB_CLIENTS__04.2007,NB_CLIENTS__05.2007,NB_CLIENTS__06.2007,NB_CLIENTS__07.2007,NB_CLIENTS__08.2007,...,KWh__05.2012,KWh__06.2012,KWh__07.2012,KWh__08.2012,KWh__09.2012,KWh__10.2012,KWh__11.2012,KWh__12.2012,KWh__Résultat,KWh__Résultat
0,agadir,Administratif,4114,4125,4179,4199,4229,4261,4274,4282,...,216955.15,4662972,183400.4,185195.48,4631808.25,185923.89,196120.19,5063733.02,20622499.58,113386824.63
1,agadir,Force_Motrice_Agricole,2019,2031,2069,2092,2124,2153,2178,2206,...,1779295,1999389.67,1749707.85,1902759.37,1750587.77,1412446.77,1149614.01,1285171.04,18930555.04,104072456.13
2,agadir,Force_Motrice_Industrielle,4038,4037,4074,4092,4109,4123,4129,4136,...,2216515,2711241,2108294,2503244,2790893,1896569,1849537,2099558,26479793,145167012
3,agadir,General,1910,1882,1873,1890,1884,1897,1914,1943,...,68852666,66140038,76264100,67571163,62208606,64240756,54885996,59948726,768529585,3984838865
4,agadir,Menages,378207,381623,384883,387704,390167,393078,396950,400579,...,53850039.73,52269874.51,56046004.84,70357822.9,60682294.99,55090631.97,57507697.2,46767296.33,645255270.38,3170711199.11
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
74,tanger,General,362,364,365,368,373,377,382,383,...,7322017,7596642,9923667,7610697,7217892,9369292,6318358,7721527,93576716,447268192
75,tanger,Menages,203676,209184,211211,198486,214334,215716,216834,203503,...,20353725.78,19035570.15,19726568.57,23865447.74,22414706.25,20705177.24,22534679.33,19434206.16,241193258.82,1179687044.46
76,tanger,Patentes,15442,15512,15566,15574,15655,15762,15832,15800,...,2523074.08,2525610.89,2715565.51,3090507.03,2894416.34,2607591.02,2544886.07,2292437.75,30188288.92,158484523.34
77,tanger,Public,1055,1055,1068,1041,1077,1096,1090,1062,...,27769.94,3771169.56,27482.01,29142.31,4397280.36,28836.87,11907.1,4731892.92,18145585.27,93562034.38


In [101]:
df_regional_orig.columns

Index(['Classe', 'Activity', 'City', 'Year', 'Month', 'Nbr Clients', 'MWh',
       'kDH', 'tempmax', 'tempmin', 'temp', 'precip', 'Region'],
      dtype='object')

In [103]:
df

,DR_DP_AS,usage,NB_CLIENTS__01.2007,NB_CLIENTS__02.2007,NB_CLIENTS__03.2007,NB_CLIENTS__04.2007,NB_CLIENTS__05.2007,NB_CLIENTS__06.2007,NB_CLIENTS__07.2007,NB_CLIENTS__08.2007,...,KWh__05.2012,KWh__06.2012,KWh__07.2012,KWh__08.2012,KWh__09.2012,KWh__10.2012,KWh__11.2012,KWh__12.2012,KWh__Résultat,KWh__Résultat
0,agadir,Administratif,4114,4125,4179,4199,4229,4261,4274,4282,...,216955.15,4662972,183400.4,185195.48,4631808.25,185923.89,196120.19,5063733.02,20622499.58,113386824.63
1,agadir,Force_Motrice_Agricole,2019,2031,2069,2092,2124,2153,2178,2206,...,1779295,1999389.67,1749707.85,1902759.37,1750587.77,1412446.77,1149614.01,1285171.04,18930555.04,104072456.13
2,agadir,Force_Motrice_Industrielle,4038,4037,4074,4092,4109,4123,4129,4136,...,2216515,2711241,2108294,2503244,2790893,1896569,1849537,2099558,26479793,145167012
3,agadir,General,1910,1882,1873,1890,1884,1897,1914,1943,...,68852666,66140038,76264100,67571163,62208606,64240756,54885996,59948726,768529585,3984838865
4,agadir,Menages,378207,381623,384883,387704,390167,393078,396950,400579,...,53850039.73,52269874.51,56046004.84,70357822.9,60682294.99,55090631.97,57507697.2,46767296.33,645255270.38,3170711199.11
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
74,tanger,General,362,364,365,368,373,377,382,383,...,7322017,7596642,9923667,7610697,7217892,9369292,6318358,7721527,93576716,447268192
75,tanger,Menages,203676,209184,211211,198486,214334,215716,216834,203503,...,20353725.78,19035570.15,19726568.57,23865447.74,22414706.25,20705177.24,22534679.33,19434206.16,241193258.82,1179687044.46
76,tanger,Patentes,15442,15512,15566,15574,15655,15762,15832,15800,...,2523074.08,2525610.89,2715565.51,3090507.03,2894416.34,2607591.02,2544886.07,2292437.75,30188288.92,158484523.34
77,tanger,Public,1055,1055,1068,1041,1077,1096,1090,1062,...,27769.94,3771169.56,27482.01,29142.31,4397280.36,28836.87,11907.1,4731892.92,18145585.27,93562034.38


In [104]:
import re
import pandas as pd

id_cols = ['DR_DP_AS', 'usage']

val_cols = [c for c in df.columns if re.match(r'^.+__\d{2}\.\d{4}$', str(c))]

long = df.melt(id_vars=id_cols, value_vars=val_cols,
               var_name='key', value_name='value')

parts = long['key'].str.extract(r'^(?P<metric>.+?)__(?P<Month>\d{2})\.(?P<Year>\d{4})')
long = pd.concat([long.drop(columns='key'), parts], axis=1)
long['Month'] = long['Month'].astype(int)
long['Year'] = long['Year'].astype(int)
long['metric'] = long['metric'].str.strip()

wide = (long.pivot_table(index=id_cols+['Year','Month'],
                         columns='metric', values='value', aggfunc='first')
             .reset_index())

wide = wide.rename(columns={'NB_CLIENTS':'Nbr Clients', 'KWh':'KWh'})

In [105]:
wide

metric,DR_DP_AS,usage,Year,Month,KWh,Nbr Clients
0,agadir,Administratif,2007,1,79412.58,4114
1,agadir,Administratif,2007,2,80575.69,4125
2,agadir,Administratif,2007,3,4337456.7,4179
3,agadir,Administratif,2007,4,83522.13,4199
4,agadir,Administratif,2007,5,86827.92,4229
...,...,...,...,...,...,...
5683,tanger,Vert,2012,8,12778960,18
5684,tanger,Vert,2012,9,7973597,19
5685,tanger,Vert,2012,10,1226471,19
5686,tanger,Vert,2012,11,152253,19


In [115]:
df_regional_orig.columns

Index(['Classe', 'Activity', 'City', 'Year', 'Month', 'Nbr Clients', 'MWh',
       'kDH', 'tempmax', 'tempmin', 'temp', 'precip', 'Region'],
      dtype='object')

In [118]:
df_regional_orig["Classe"].unique()

array(['BT/PP', 'BT', 'MT'], dtype=object)

In [120]:
wide["usage"].unique()

array(['Administratif', 'Force_Motrice_Agricole',
       'Force_Motrice_Industrielle', 'General', 'Menages', 'Patentes',
       'Public', 'Vert'], dtype=object)

In [123]:
wide["Classe"] = wide["usage"].apply(lambda x: "MT" if x in ["General", "Vert"] else "BT")

In [124]:
wide.columns

Index(['DR_DP_AS', 'usage', 'Year', 'Month', 'KWh', 'Nbr Clients', 'Classe'], dtype='object', name='metric')

In [ ]:
wide["KWh"] = wide["KWh"] / 1000

In [127]:
wide.rename(columns = {"usage":"Activity", "DR_DP_AS":"City", "KWh": "MWh"}, inplace = True)

In [130]:
city_to_region = {
    'beni_mellal': 'Béni Mellal-Khénifra',
    'casablanca': 'Casablanca-Settat',
    'errachidia': 'Drâa-Tafilalet',
    'fes': 'Fès-Meknès',
    'laayoune': 'Laâyoune-Sakia El Hamra',
    'marrakech': 'Marrakech-Safi',
    'oujda': 'Oriental',
    'rabat': 'Rabat-Salé-Kénitra',
    'agadir': 'Souss-Massa',
    'tanger': 'Tanger-Tétouan-Al Hoceïma'
}


In [131]:
wide["Region"] = wide["City"].apply(lambda x: city_to_region[x])

In [151]:
wide.columns

Index(['City', 'Activity', 'Year', 'Month', 'MWh', 'Nbr Clients', 'Classe',
       'Region'],
      dtype='object', name='metric')

In [150]:
df_regional_orig.columns

Index(['Classe', 'Activity', 'City', 'Year', 'Month', 'Nbr Clients', 'MWh',
       'kDH', 'tempmax', 'tempmin', 'temp', 'precip', 'Region'],
      dtype='object')

In [158]:
final_df = pd.concat([df_regional_orig, wide]).sort_values(by = ["Year", "Month"])

In [160]:
final_df.to_csv(PROJECT_ROOT / "data/srm_data_2007_2023.csv", index = False)

In [4]:
final_df = pd.read_csv(PROJECT_ROOT / "data/srm_data_2007_2023.csv")

In [9]:
final_df = final_df[final_df["Classe"] != "MT"]

In [11]:
import sqlite3
import pandas as pd
from pathlib import Path

def clone_db(db_orig: sqlite3.Connection, new_path: str) -> sqlite3.Connection:
    new_path = Path(new_path)
    if new_path.exists():
        new_path.unlink()
    db_new = sqlite3.connect(new_path.as_posix())
    db_orig.backup(db_new)
    db_new.execute("PRAGMA foreign_keys = ON;")
    return db_new

def replace_table_with_df(db_orig: sqlite3.Connection, df: pd.DataFrame, table: str, new_db_path: str):
    db_new = clone_db(db_orig, new_db_path)
    try:
        db_new.execute("BEGIN;")
        db_new.execute(f"DROP TABLE IF EXISTS monthly_data;")
        # This drops the old table and recreates a plain one (no PK/FKs/indexes/triggers).
        df.to_sql(table, db_new, if_exists="replace", index=False)
        db_new.commit()
    except Exception:
        db_new.rollback()
        raise
    finally:
        db_new.close()

# usage:
replace_table_with_df(db_orig, final_df, "monthly_data_bt", PROJECT_ROOT / "data/ONEE_Regional_COMPLETE_2007_2023.db")

In [12]:
db_orig.close()